# Многопоточность в Python. Библиотеки threading и multiprocessing.

http://cs.mipt.ru/advanced_python/lessons/lab6.html

## Задание 1:

распараллелить чистку текста (колонки text). Функция чистки текста уже написана за вас. Вам нужно только установить все библиотеки (например, через анаконду)


https://anaconda.org/anaconda/nltk

In [1]:
# подрубаем нужные библиотеки
import os # для путей
import numpy as np # для массивов
import pandas as pd # для работы и хранения данных в ОП

import warnings 
warnings.filterwarnings('ignore') # все предупреждения офф - чтобы не мешали

from nltk.corpus import stopwords # частые слова
from nltk.stem import PorterStemmer # нлтк стеммер
from nltk.stem import WordNetLemmatizer # нлтк лемматизатор
from nltk import word_tokenize, sent_tokenize # для разбиения предложений на токены-слова
import re # библиотека для регулярных выражений

данные здесь: https://www.kaggle.com/c/dlinnlp-spring-2019-clf/data

In [14]:
path = 'C:/Users/user/kaggle_competitions/news_classification3_ipavlov' # изменить на нужный путь к файлу
train_df = pd.read_csv (os.path.join(path, 'train.csv'))
test_df = pd.read_csv (os.path.join(path, 'test.csv'))
df = pd.concat([train_df, test_df], axis=0, ignore_index=True).drop(['Unnamed: 0', 'label'], axis=1)
print(len(train_df), len(test_df), len(df))
df.head(10)

24871 5647 30518


,text,title
0,Economists generally agree: China must overhau...,China and Economic Reform: Xi Jinping’s Track ...
1,LONDON—British Prime Minister Theresa May said...,Trade to Be a Big Topic in Theresa May’s U.S. ...
2,Beaches come in all sorts of shapes and sizes ...,"The Top Beaches In The World, According To Nat..."
3,"A timeline of what happened after Tamir Rice, ...",Sheriff’s Report Provides New Details on Tamir...
4,An Italian neurosurgeon who has claimed for mo...,Surgeon claiming he will transplant volunteer'...
5,Spanx does not do the thing I thought it did. ...,This Is How Differently Priced Spanx Can Actua...
6,"“Jane Pauley! You’re all in leather,” said Sam...",Samantha Bee and Jane Pauley Are Breaking the ...
7,Charles Krauthammer called President Trump's 5...,Krauthammer: Syria Strike 'Total Contradiction...
8,The United States is making more things than e...,Rust Belt voters made Trump president. Now the...
9,As Jewish settlers and protesters were removed...,"As Illegal Outpost Cleared, Israeli PM Netanya..."


In [16]:
# функция для очистки текста

def clean_text(data):
    low = True 
    delstops = True
    lemm = True
    stem = True
    del12gram = True  # True = убираем обрывки слов в 1-2 символа
    
    
    stops = set(stopwords.words("english"))
    data = str(data)
    data = re.sub(r'[^a-zA-Z ]',r' ',data)
    data = ' '.join(data.split()) #убираем лишние пробелы
    
    if low:
        data = data.lower()
    
    if delstops:
        data = " ".join([w for w in data.split() if w not in stops])

    if lemm:      
        wordnet_lemmatizer = WordNetLemmatizer()
        data = " ".join([wordnet_lemmatizer.lemmatize(w, pos='a') for w in data.split()])
        data = " ".join([wordnet_lemmatizer.lemmatize(w, pos='n') for w in data.split()])
        
    if stem:
        st = PorterStemmer()
        data = " ".join([st.stem(w) for w in data.split()])
        
    if del12gram:
        data = " ".join([w for w in data.split() if len(w) > 2])  
          
    return data

In [17]:
%%time
# применяем функцию очистки текста
df['cleaned_text'] = df['text'].map(clean_text) # ~ Wall time: 5min 34s

Wall time: 5min 34s


In [18]:
df.head(10)

,text,title,cleaned_text
0,Economists generally agree: China must overhau...,China and Economic Reform: Xi Jinping’s Track ...,economist gener agre china must overhaul huge ...
1,LONDON—British Prime Minister Theresa May said...,Trade to Be a Big Topic in Theresa May’s U.S. ...,london british prime minist theresa may said d...
2,Beaches come in all sorts of shapes and sizes ...,"The Top Beaches In The World, According To Nat...",beach come sort shape size beyond typic caribb...
3,"A timeline of what happened after Tamir Rice, ...",Sheriff’s Report Provides New Details on Tamir...,timelin happen tamir rice year old boy kill po...
4,An Italian neurosurgeon who has claimed for mo...,Surgeon claiming he will transplant volunteer'...,italian neurosurgeon claim month perform world...
5,Spanx does not do the thing I thought it did. ...,This Is How Differently Priced Spanx Can Actua...,spanx thing thought kristin like mani ladi lil...
6,"“Jane Pauley! You’re all in leather,” said Sam...",Samantha Bee and Jane Pauley Are Breaking the ...,jane pauley leather said samantha bee pauley w...
7,Charles Krauthammer called President Trump's 5...,Krauthammer: Syria Strike 'Total Contradiction...,charl krauthamm call presid trump missil strik...
8,The United States is making more things than e...,Rust Belt voters made Trump president. Now the...,unit state make thing ever read right manufact...
9,As Jewish settlers and protesters were removed...,"As Illegal Outpost Cleared, Israeli PM Netanya...",jewish settler protest remov illeg outpost occ...


# Задание 2

Иногда бывает нужно узнать доступность набора ip адресов. Неэффективный вариант представлен ниже.

Реализуйте то же самое, но используя threading.

In [2]:
import os, re

received_packages = re.compile(r"(\d) received")
status = ("no response", "alive but losses", "alive")

for suffix in range(20, 30):
    ip = "192.168.178." + str(suffix)
    ping_out = os.popen("ping -q -c2 " + ip, "r")  # получение вердикта
    print("... pinging ", ip)
    while True:
        line = ping_out.readline()
        if not line:
            break
        n_received = received_packages.findall(line)
        if n_received:
            print(ip + ": " + status[int(n_received[0])])

... pinging  192.168.178.20
... pinging  192.168.178.21
... pinging  192.168.178.22
... pinging  192.168.178.23
... pinging  192.168.178.24
... pinging  192.168.178.25
... pinging  192.168.178.26
... pinging  192.168.178.27
... pinging  192.168.178.28
... pinging  192.168.178.29


In [1]:
import http.server
import socketserver
from threading import Thread
import os

def server_thread(port):
    handler = http.server.SimpleHTTPRequestHandler
    with socketserver.TCPServer(("", port), handler) as httpd:
        httpd.serve_forever()

if __name__ == '__main__':

    port = 8000
    print("Starting server at port %d" % port)
    os.chdir("/home/pi/Documents")
    Thread(target=server_thread, args=(port,)).start()

Starting server at port 8000


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: '/home/pi/Documents'

# Задание 3

= Упражнение №4 из лабы про мультипроцессинг

http://cs.mipt.ru/advanced_python/lessons/lab6.html#id14

In [ ]:
import urllib.request
import time


urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
for url in urls:
    read_url(url)